## OMS Agent with ADK
https://github.com/ra2085/apigee-adk-workshop.git
<table align="left">
    <td style="text-align: center">
        <a href="https://colab.research.google.com/github/ra2085/apigee-adk-workshop/blob/main/notebooks/oms-agent.ipynb">
          <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo\"><br> Open in Colab
        </a>
      </td>
      <td style="text-align: center">
        <a href="https://console.cloud.google.com/vertex-ai/colab/import/https%3A%2F%2Fraw.githubusercontent.com%2Fra2085%2Fapigee-adk-workshop%2Fmain%2Fnotebooks%2Foms-agent.ipynb">
          <img width="32px" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" alt="Google Cloud Colab Enterprise logo"><br> Open in Colab Enterprise
        </a>
      </td>    
      <td style="text-align: center">
        <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/ra2085/apigee-adk-workshop/main/notebooks/oms-agent.ipynb">
          <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo"><br> Open in Workbench
        </a>
      </td>
      <td style="text-align: center">
        <a href="https://github.com/ra2085/apigee-adk-workshop/blob/main/notebooks/oms-agent.ipynb">
          <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo"><br> View on GitHub
        </a>
      </td>
</table>
<br />
<br />
<br />
<br />

### Install Dependencies

In [ ]:
%pip install google-adk -q
%pip install litellm -q

### Import Dependencies

In [ ]:
import os
from dotenv import load_dotenv
from google.genai import types
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools.application_integration_tool.application_integration_toolset import ApplicationIntegrationToolset
from google.adk.tools.openapi_tool.auth.auth_helpers import token_to_scheme_credential
from google.adk.tools.apihub_tool.apihub_toolset import APIHubToolset
from fastapi.openapi.models import OAuth2
from fastapi.openapi.models import OAuthFlowClientCredentials
from fastapi.openapi.models import OAuthFlows
from google.adk.auth import AuthCredential
from google.adk.auth import AuthCredentialTypes
from google.adk.auth import OAuth2Auth


In [ ]:
#os.environ["GOOGLE_API_KEY"] = "" # <--- REPLACE
os.environ["GOOGLE_GENAI_USE_VERTEXAI"]=""
os.environ["GOOGLE_CLOUD_PROJECT"]=""
os.environ["GOOGLE_CLOUD_LOCATION"]=""

### Initialize API Hub Tools

In [ ]:
async def get_product_tools_async():
    """Gets CRM from API Hub."""
    auth_scheme = OAuth2(
        flows=OAuthFlows(
            clientCredentials=OAuthFlowClientCredentials(
                tokenUrl="https://APIGEE_HOST/mcp/token", # @param {type:"string"}
                scopes={},
                
            )
        )
    )
    auth_credential = AuthCredential(
        auth_type=AuthCredentialTypes.OAUTH2,
        oauth2=OAuth2Auth(
            client_id="YOUR_OAUTH_CLIENT_ID", # @param {type:"string"}
            client_secret="YOUR_OAUTH_CLIENT_SECRET" # @param {type:"string"}
        ),
    )

    connector_tool = ApplicationIntegrationToolset(
        project="", 
        location="us-central1",
        connection="oms-connection", 
        entity_operations={"Entity_One": ["LIST"], "Entity_Two": []},#empty list for actions means all operations on the entity are supported.
        actions=["action1"],
        service_account_credentials='{...}', # optional
        tool_name="products_get",
        tool_instructions="Gproducts and product details"
    )
    tools = connector_tool.get_tools()
    return tools